<a href="https://colab.research.google.com/github/sungjk1999/smart-facotry-quality-classification/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn

print('test')
print('commit')

test
